In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from hwsnn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from hwsnn.tonic_dataloader import DatasetLoader
from hwsnn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'shd_soa_baselines'

data = test_loader, train_loader

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

In [7]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]


## Baseline (no dropout) -> 87.5

In [8]:
train(snn, data, 1e-4, 50, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, scheduler=(1, 0.98))

RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49123
Step [20/31], Loss: 0.47768
Step [30/31], Loss: 0.45534
Time elasped: 39.090970516204834
Epoch [2/50]
Step [10/31], Loss: 0.42956
Step [20/31], Loss: 0.41055
Step [30/31], Loss: 0.39110
Time elasped: 39.346556663513184
Epoch [3/50]
Step [10/31], Loss: 0.37323
Step [20/31], Loss: 0.35470
Step [30/31], Loss: 0.33961
Time elasped: 39.052327156066895
Epoch [4/50]
Step [10/31], Loss: 0.32846
Step [20/31], Loss: 0.31343
Step [30/31], Loss: 0.30237
Time elasped: 38.97873544692993
Epoch [5/50]
Step [10/31], Loss: 0.29187
Step [20/31], Loss: 0.28027
Step [30/31], Loss: 0.27503
Time elasped: 39.00328063964844
Test Loss: 0.031483145164

## Baseline (dropout = 0.2) -> 87.1

In [7]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49450
Step [20/31], Loss: 0.47842
Step [30/31], Loss: 0.46485
Time elasped: 56.74550414085388
Epoch [2/50]
Step [10/31], Loss: 0.44598
Step [20/31], Loss: 0.42368
Step [30/31], Loss: 0.40241
Time elasped: 56.640833616256714
Epoch [3/50]
Step [10/31], Loss: 0.38519
Step [20/31], Loss: 0.36767
Step [30/31], Loss: 0.35211
Time elasped: 56.34934186935425
Epoch [4/50]
Step [10/31], Loss: 0.34113
Step [20/31], Loss: 0.3

## 64_2 150,5 -> 85.05

In [3]:
hidden = (64, 2, 'f')
depth, stride = 150, 5
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=1920, out_features=64, bias=False)
  (f2_o): Linear(in_features=1920, out_features=20, bias=False)
)
Total params: 206080
Total mult-adds (M): 51.52
training shd250_64x2_d_150_5_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49800
Step [20/31], Loss: 0.48775
Step [30/31], Loss: 0.47364
Time elasped: 61.0452241897583
Epoch [2/50]
Step [10/31], Loss: 0.45639
Step [20/31], Loss: 0.44041
Step [30/31], Loss: 0.42392
Time elasped: 55.56919026374817
Epoch [3/50]
Step [10/31], Loss: 0.40782
Step [20/31], Loss: 0.39555
Step [30/31], Loss: 0.38365
Time elasped: 54.820889949798584
Epoch [4/50]
Step [10/31], Loss: 0.36995
Step [20/31], Loss: 0.36351
Step [30/31], Loss: 0.35341
Time elasped: 54.9261794090271
Epoch [5/50]
Step [1

In [8]:
hidden = (64, 2, 'f')
depth, stride = 150, 10
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=960, out_features=64, bias=False)
  (f2_o): Linear(in_features=960, out_features=20, bias=False)
)
Total params: 125440
Total mult-adds (M): 31.36
training shd250_64x2_d_150_10_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49694
Step [20/31], Loss: 0.49265
Step [30/31], Loss: 0.48200
Time elasped: 56.201778411865234
Epoch [2/50]
Step [10/31], Loss: 0.47152
Step [20/31], Loss: 0.46016
Step [30/31], Loss: 0.44966
Time elasped: 56.030195236206055
Epoch [3/50]
Step [10/31], Loss: 0.43665
Step [20/31], Loss: 0.42846
Step [30/31], Loss: 0.41768
Time elasped: 56.044498920440674
Epoch [4/50]
Step [10/31], Loss: 0.40713
Step [20/31], Loss: 0.39930
Step [30/31], Loss: 0.39240
Time elasped: 55.81705856323242
Epoch [5/50]
Step [10/31], Loss: 0.38514
Step [20/31], Loss: 0.37752
Step [30/31]

In [4]:
hidden = (64, 1, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 108800
Total mult-adds (M): 27.2
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49000
Step [20/31], Loss: 0.48147
Step [30/31], Loss: 0.47101
Time elasped: 41.75855255126953
Epoch [2/50]
Step [10/31], Loss: 0.45880
Step [20/31], Loss: 0.45086
Step [30/31], Loss: 0.44129
Time elasped: 41.69176268577576
Epoch [3/50]
Step [10/31], Loss: 0.43141
Step [20/31], Loss: 0.42558
Step [30/31], Loss: 0.41914
Time elasped: 42.452242374420166
Epoch [4/50]
Step [10/31], Loss: 0.41088
Step [20/31], Loss: 0.40425
Step [30/31], Loss: 0.40117
Time elasped: 41.748841762542725

In [5]:
hidden = (64, 1, 'f')
depth, stride = 150, 1
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_o): Linear(in_features=9600, out_features=20, bias=False)
)
Total params: 236800
Total mult-adds (M): 59.2
training shd250_64x2_d_150_1_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.48913
Step [20/31], Loss: 0.47254
Step [30/31], Loss: 

In [6]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49798
Step [20/31], Loss: 0.49563
Step [30/31], Loss: 0.49065
Time elasped: 64.85917925834656
Epoch [2/50]
Step [10/31], Loss: 0.48644
Step [20/31], Loss: 0.48109
Step [30/31], Loss: 0.47470
Time elasped: 64.47341179847717
Epoch [3/50]
Step [10/31], Loss: 0.46571
Step [20/31], Loss: 0.45884
Step [30/31], Loss: 0.45161
Time elasped: 63.80383253097534
Epoch [4/50]
Step [10/31], Loss: 0.44444
Step [20/31], Loss: 0.4371

In [7]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'mg'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 0.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_mg for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.50153
Step [20/31], Loss: 0.48948
Step [30/31], Loss: 0.48074
Time elasped: 60.54729127883911
Epoch [2/50]
Step [10/31], Loss: 0.47252
Step [20/31], Loss: 0.46616
Step [30/31], Loss: 0.45785
Time elasped: 60.43790531158447
Epoch [3/50]
Step [10/31], Loss: 0.44824
Step [20/31], Loss: 0.43621
Step [30/31], Loss: 0.42498
Time elasped: 60.37019872665405
Epoch [4/50]
Step [10/31], Loss: 0.41587
Step [20/31], Loss: 0.4087

### Tests with M-O-T

In [9]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.36632
Step [20/31], Loss: 27.13333
Step [30/31], Loss: 23.19125
Time elasped: 61.1243097782135
Epoch [2/50]
Step [10/31], Loss: 16.82194
Step [20/31], Loss: 13.46069
Step [30/31], Loss: 11.29933
Time elasped: 58.900102853775024
Epoch [3/50]
Step [10/31], Loss: 9.45210
Step [20/31], Loss: 8.13538
Step [30/31], Loss: 7.32290
Time elasped: 58.62159991264343
Epoch [4/50]
Step [10/31], Loss: 6.55422
Step [20/

In [10]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.57361
Step [20/31], Loss: 27.95540
Step [30/31], Loss: 24.47157
Time elasped: 66.59746742248535
Epoch [2/50]
Step [10/31], Loss: 19.07831
Step [20/31], Loss: 14.81110
Step [30/31], Loss: 12.47497
Time elasped: 65.15126085281372
Epoch [3/50]
Step [10/31], Loss: 10.25926
Step [20/31], Loss: 8.84390
Step [30/31], Loss: 8.44268
Time elasped: 65.39317727088928
Epoch [4/50]
Step [10/31], Loss: 7.18084
Step [20/3

In [11]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'mg'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_mg for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.63929
Step [20/31], Loss: 28.28706
Step [30/31], Loss: 26.42651
Time elasped: 64.23737835884094
Epoch [2/50]
Step [10/31], Loss: 22.92960
Step [20/31], Loss: 18.96483
Step [30/31], Loss: 16.14996
Time elasped: 61.9026780128479
Epoch [3/50]
Step [10/31], Loss: 13.71708
Step [20/31], Loss: 11.45429
Step [30/31], Loss: 10.20832
Time elasped: 61.77303695678711
Epoch [4/50]
Step [10/31], Loss: 9.06298
Step [20/

In [14]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'mg'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_mg for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.36641
Step [20/31], Loss: 28.09900
Step [30/31], Loss: 26.19979
Time elasped: 60.481040716171265
Epoch [2/50]
Step [10/31], Loss: 22.80946
Step [20/31], Loss: 18.53254
Step [30/31], Loss: 15.24051
Time elasped: 60.6999077796936
Epoch [3/50]
Step [10/31], Loss: 11.59477
Step [20/31], Loss: 10.64960
Step [30/31], Loss: 9.77517
Time elasped: 60.633605003356934
Epoch [4/50]
Step [10/31], Loss: 8.50842
Step [20

In [12]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'mg'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_mg for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.55384
Step [20/31], Loss: 28.60849
Step [30/31], Loss: 27.04680
Time elasped: 61.380842208862305
Epoch [2/50]
Step [10/31], Loss: 23.64129
Step [20/31], Loss: 19.62600
Step [30/31], Loss: 15.94421
Time elasped: 61.62552237510681
Epoch [3/50]
Step [10/31], Loss: 13.72862
Step [20/31], Loss: 11.85832
Step [30/31], Loss: 10.05791
Time elasped: 61.14820694923401
Epoch [4/50]
Step [10/31], Loss: 8.61640
Step [2

#### mg-mot: 88.57

In [13]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'mg'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_mg for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 27.78429
Step [20/31], Loss: 18.82737
Step [30/31], Loss: 12.80479
Time elasped: 60.72176384925842
Epoch [2/50]
Step [10/31], Loss: 8.69751
Step [20/31], Loss: 7.56100
Step [30/31], Loss: 5.80616
Time elasped: 60.599273681640625
Epoch [3/50]
Step [10/31], Loss: 4.14722
Step [20/31], Loss: 3.90162
Step [30/31], Loss: 3.40539
Time elasped: 60.50635027885437
Epoch [4/50]
Step [10/31], Loss: 2.54115
Step [20/31],

In [3]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.23731
Step [20/31], Loss: 21.47617
Step [30/31], Loss: 13.75138
Time elasped: 58.22463321685791
Epoch [2/50]
Step [10/31], Loss: 9.44397
Step [20/31], Loss: 7.27643
Step [30/31], Loss: 5.32362
Time elasped: 58.11188268661499
Epoch [3/50]
Step [10/31], Loss: 4.48906
Step [20/31], Loss: 3.51091
Step [30/31], Loss: 3.28651
Time elasped: 58.62482023239136
Epoch [4/50]
Step [10/31], Loss: 2.81387
Step [20/31]

In [4]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 27.94047
Step [20/31], Loss: 18.97810
Step [30/31], Loss: 12.93364
Time elasped: 59.1649227142334
Epoch [2/50]
Step [10/31], Loss: 8.84668
Step [20/31], Loss: 6.90321
Step [30/31], Loss: 5.91141
Time elasped: 59.003658056259155
Epoch [3/50]
Step [10/31], Loss: 4.66196
Step [20/31], Loss: 4.40300
Step [30/31], Loss: 2.95821
Time elasped: 59.034255027770996
Epoch [4/50]
Step [10/31], Loss: 2.17653
Step [20/31],

In [2]:
hidden = (64, 2, 'f')
depth, stride = 150, 1
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=9600, out_features=64, bias=False)
  (f2_o): Linear(in_features=9600, out_features=20, bias=False)
)
Total params: 851200
Total mult-adds (M): 212.8
training shd250_64x2_d_150_1_fs for 50 epochs...
Epoch [1/50]
Ste

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 12.00 GiB total capacity; 9.97 GiB already allocated; 0 bytes free; 10.61 GiB reserved in total by PyTorch)

# BASELINES

#### fs-mot: 89.8

In [2]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.31540
Step [20/31], Loss: 23.47349
Step [30/31], Loss: 14.43590
Time elasped: 71.47801470756531
Epoch [2/50]
Step [10/31], Loss: 9.64823
Step [20/31], Loss: 7.62114
Step [30/31], Loss: 5.71666
Time elasped: 65.41291427612305
Epoch [3/50]
Step [10/31], Loss: 4.61027
Step [20/31], Loss: 3.83731
Step [30/31], Loss: 3.35590
Time elasped: 65.29130625724792
Epoch [4/50]
Step [10/31], Loss: 2.81150
Step [20/31], 

#### fs-mot (150,6): 89.014

In [2]:
hidden = (64, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=1600, out_features=64, bias=False)
  (f2_o): Linear(in_features=1600, out_features=20, bias=False)
)
Total params: 179200
Total mult-adds (M): 44.8
training shd250_64x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.24467
Step [20/31], Loss: 20.79190
Step [30/31], Loss: 13.52454
Time elasped: 68.56922101974487
Epoch [2/50]
Step [10/31], Loss: 8.24595
Step [20/31], Loss: 6.13252
Step [30/31], Loss: 5.23191
Time elasped: 62.62478947639465
Epoch [3/50]
Step [10/31], Loss: 4.05835
Step [20/31], Loss: 3.71178
Step [30/31], Loss: 3.00914
Time elasped: 61.958678245544434
Epoch [4/50]
Step [10/31], Loss: 2.44121
Step [20/31], Loss: 2.12166
Step [30/31], Loss: 2.27886
Time elasped: 61.997377157211304
Epoch [5/50]
Step [10/31], Los

#### fs-mot-128: 86.71

In [2]:
hidden = (128, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=128, bias=False)
  (f1_f2): Linear(in_features=6400, out_features=128, bias=False)
  (f2_o): Linear(in_features=6400, out_features=20, bias=False)
)
Total params: 1036800
Total mult-adds (M): 259.2
training shd250_128x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.91467
Step [20/31], Loss: 21.97455
Step [30/31], Loss: 14.04255
Time elasped: 91.37040400505066
Epoch [2/50]
Step [10/31], Loss: 9.16258
Step [20/31], Loss: 7.46027
Step [30/31], Loss: 5.59668
Time elasped: 86.4264349937439
Epoch [3/50]
Step [10/31], Loss: 4.79026
Step [20/31], Loss: 3.40417
Step [30/31], Loss: 2.54557
Time elasped: 85.055184841156
Epoch [4/50]
Step [10/31], Loss: 2.08343
Step [20/31]

#### fs-mot-128 (150,6): 89.45

In [3]:
hidden = (128, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=128, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=128, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 563200
Total mult-adds (M): 140.8
training shd250_128x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.10148
Step [20/31], Loss: 20.96344
Step [30/31], Loss: 13.98017
Time elasped: 80.87059831619263
Epoch [2/50]
Step [10/31], Loss: 9.21802
Step [20/31], Loss: 6.88492
Step [30/31], Loss: 4.98712
Time elasped: 80.09560132026672
Epoch [3/50]
Step [10/31], Loss: 3.49544
Step [20/31], Loss: 3.21454
Step [30/31], Loss: 2.94330
Time elasped: 78.81742238998413
Epoch [4/50]
Step [10/31], Loss: 2.07641
Step [20/31], Loss: 1.75559
Step [30/31], Loss: 1.91796
Time elasped: 80.04842615127563
Epoch [5/50]
Step [10/31], L

#### fs-mot-48 (150,10)

In [2]:
hidden = (48, 2, 'f')
depth, stride = 150, 10
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=720, out_features=48, bias=False)
  (f2_o): Linear(in_features=720, out_features=20, bias=False)
)
Total params: 82560
Total mult-adds (M): 20.64
training shd250_48x2_d_150_10_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.94742
Step [20/31], Loss: 22.84978
Step [30/31], Loss: 14.32422
Time elasped: 62.20410370826721
Epoch [2/50]
Step [10/31], Loss: 9.57610
Step [20/31], Loss: 6.77060
Step [30/31], Loss: 5.65064
Time elasped: 58.082245111465454
Epoch [3/50]
Step [10/31], Loss: 4.19838
Step [20/31], Loss: 4.04935
Step [30/31], Loss: 3.71158
Time elasped: 58.26968312263489
Epoch [4/50]
Step [10/31], Loss: 3.23092
Step [20/31], Loss: 2.78503
Step [30/31], Loss: 2.25324
Time elasped: 58.298075914382935
Epoch [5/50]
Step [10/31], Loss: 2.05825
Step [20/31], Loss: 2.17937
Step

#### fs-mot-64 (150,10)

In [3]:
hidden = (64, 2, 'f')
depth, stride = 150, 10
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=960, out_features=64, bias=False)
  (f2_o): Linear(in_features=960, out_features=20, bias=False)
)
Total params: 125440
Total mult-adds (M): 31.36
training shd250_64x2_d_150_10_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.04106
Step [20/31], Loss: 19.90625
Step [30/31], Loss: 13.22229
Time elasped: 60.05324196815491
Epoch [2/50]
Step [10/31], Loss: 9.01291
Step [20/31], Loss: 6.98595
Step [30/31], Loss: 5.28808
Time elasped: 60.07624316215515
Epoch [3/50]
Step [10/31], Loss: 4.36443
Step [20/31], Loss: 4.02771
Step [30/31], Loss: 3.21505
Time elasped: 60.000319480895996
Epoch [4/50]
Step [10/31], Loss: 2.45074
Step [20/31], Loss: 2.91211
Step [30/31], Loss: 2.45185
Time elasped: 60.09979271888733
Epoch [5/50]
Step [10/31], Loss: 1.90687
Step [20/31], Loss: 2.23202
Step

#### fs-mot-128 (150,10)

In [4]:
hidden = (128, 2, 'f')
depth, stride = 150, 10
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=128, bias=False)
  (f1_f2): Linear(in_features=1920, out_features=128, bias=False)
  (f2_o): Linear(in_features=1920, out_features=20, bias=False)
)
Total params: 373760
Total mult-adds (M): 93.44
training shd250_128x2_d_150_10_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.54579
Step [20/31], Loss: 20.46967
Step [30/31], Loss: 12.89911
Time elasped: 78.21185183525085
Epoch [2/50]
Step [10/31], Loss: 8.63105
Step [20/31], Loss: 6.37820
Step [30/31], Loss: 4.91432
Time elasped: 76.99223470687866
Epoch [3/50]
Step [10/31], Loss: 4.57820
Step [20/31], Loss: 3.36747
Step [30/31], Loss: 3.24082
Time elasped: 76.99347281455994
Epoch [4/50]
Step [10/31], Loss: 2.63937
Step [20/31], Loss: 1.98025
Step [30/31], Loss: 2.01475
Time elasped: 77.03213381767273
Epoch [5/50]
Step [10/31], Loss: 1.55597
Step [20/31], Loss: 1.51876


#### fs-mot-16 (150,3)

In [2]:
hidden = (16, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=800, out_features=16, bias=False)
  (f2_o): Linear(in_features=800, out_features=20, bias=False)
)
Total params: 40000
Total mult-adds (M): 10.0
training shd250_16x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.26088
Step [20/31], Loss: 25.59121
Step [30/31], Loss: 20.27267
Time elasped: 73.14639496803284
Epoch [2/50]
Step [10/31], Loss: 15.03308
Step [20/31], Loss: 11.98285
Step [30/31], Loss: 9.12671
Time elasped: 55.498929500579834
Epoch [3/50]
Step [10/31], Loss: 7.04461
Step [20/31], Loss: 6.91604
Step [30/31], Loss: 6.49634
Time elasped: 55.30768346786499
Epoch [4/50]
Step [10/31], Loss: 5.06694
Step [20/31], 

KeyboardInterrupt: 

#### fs-mot-16 (150,6)

In [3]:
hidden = (16, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=400, out_features=16, bias=False)
  (f2_o): Linear(in_features=400, out_features=20, bias=False)
)
Total params: 25600
Total mult-adds (M): 6.4
training shd250_16x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.12324
Step [20/31], Loss: 24.41766
Step [30/31], Loss: 18.29039
Time elasped: 55.43080019950867
Epoch [2/50]
Step [10/31], Loss: 13.14109
Step [20/31], Loss: 10.14471
Step [30/31], Loss: 8.78458
Time elasped: 54.718241453170776
Epoch [3/50]
Step [10/31], Loss: 7.28501
Step [20/31], Loss: 6.17582
Step [30/31], Loss: 5.07319
Time elasped: 55.96347641944885
Epoch [4/50]
Step [10/31], Loss: 4.85311
Step [20/31], Loss: 4.30888
Step [30/31], Loss: 4.19416
Time elasped: 55.06698775291443
Epoch [5/50]
Step [10/31], Loss: 

In [4]:
hidden = (16, 2, 'f')
depth, stride = 150, 15
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 15, 30, 45, 60, 75, 90, 105, 120, 135]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=160, out_features=16, bias=False)
  (f2_o): Linear(in_features=160, out_features=20, bias=False)
)
Total params: 16960
Total mult-adds (M): 4.24
training shd250_16x2_d_150_15_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.13592
Step [20/31], Loss: 26.67370
Step [30/31], Loss: 23.36497
Time elasped: 54.97896480560303
Epoch [2/50]
Step [10/31], Loss: 18.81187
Step [20/31], Loss: 14.83901
Step [30/31], Loss: 11.45581
Time elasped: 54.42464590072632
Epoch [3/50]
Step [10/31], Loss: 8.96419
Step [20/31], Loss: 8.07924
Step [30/31], Loss: 7.13460
Time elasped: 54.46797180175781
Epoch [4/50]
Step [10/31], Loss: 6.60215
Step [20/31], Loss: 5.55888
Step [30/31], Loss: 5.53953
Time elasped: 54.25066804885864
Epoch [5/50]
Step [10/31], Loss: 5.11896
Step [20/31], Loss: 4.54972
Step [30/31], Loss: 4.5659

In [5]:
hidden = (16, 2, 'f')
depth, stride = 150, 10
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=240, out_features=16, bias=False)
  (f2_o): Linear(in_features=240, out_features=20, bias=False)
)
Total params: 19840
Total mult-adds (M): 4.96
training shd250_16x2_d_150_10_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.65098
Step [20/31], Loss: 27.41792
Step [30/31], Loss: 23.22327
Time elasped: 53.881006956100464
Epoch [2/50]
Step [10/31], Loss: 18.12108
Step [20/31], Loss: 13.93254
Step [30/31], Loss: 11.27298
Time elasped: 53.91172432899475
Epoch [3/50]
Step [10/31], Loss: 9.31504
Step [20/31], Loss: 7.93131
Step [30/31], Loss: 6.47339
Time elasped: 54.06983780860901
Epoch [4/50]
Step [10/31], Loss: 6.06389
Step [20/31], Loss: 5.57760
Step [30/31], Loss: 4.83419
Time elasped: 53.91684079170227
Epoch [5/50]
Step [10/31], Loss: 4.08336
Step [20/31], Loss: 4.34658
Ste

In [6]:
hidden = (16, 2, 'f')
depth, stride = 150, 30
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 30, 60, 90, 120]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=80, out_features=16, bias=False)
  (f2_o): Linear(in_features=80, out_features=20, bias=False)
)
Total params: 14080
Total mult-adds (M): 3.52
training shd250_16x2_d_150_30_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.57811
Step [20/31], Loss: 28.48640
Step [30/31], Loss: 26.89550
Time elasped: 73.92824983596802
Epoch [2/50]
Step [10/31], Loss: 24.16483
Step [20/31], Loss: 21.52505
Step [30/31], Loss: 18.27154
Time elasped: 58.3118052482605
Epoch [3/50]
Step [10/31], Loss: 15.49108
Step [20/31], Loss: 13.37325
Step [30/31], Loss: 12.01553
Time elasped: 58.83877086639404
Epoch [4/50]
Step [10/31], Loss: 10.34920
Step [20/31], Loss: 9.60650
Step [30/31], Loss: 8.60448
Time elasped: 55.83959150314331
Epoch [5/50]
Step [10/31], Loss: 7.83127
Step [20/31], Loss: 7.44827
Step [30/31], Loss: 7.29131
Time elasped: 55.07

In [15]:
hidden = (16, 2, 'f')
depth, stride = 151, 75
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 75, 150]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=16, bias=False)
  (f1_f2): Linear(in_features=48, out_features=16, bias=False)
  (f2_o): Linear(in_features=48, out_features=20, bias=False)
)
Total params: 12928
Total mult-adds (M): 3.232
training shd250_16x2_d_151_75_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.95588
Step [20/31], Loss: 29.31661
Step [30/31], Loss: 28.54742
Time elasped: 54.99985313415527
Epoch [2/50]
Step [10/31], Loss: 27.01438
Step [20/31], Loss: 25.35861
Step [30/31], Loss: 23.87513
Time elasped: 54.336241722106934
Epoch [3/50]
Step [10/31], Loss: 21.76751
Step [20/31], Loss: 19.73032
Step [30/31], Loss: 17.87101
Time elasped: 56.75087594985962
Epoch [4/50]
Step [10/31], Loss: 15.51125
Step [20/31], Loss: 13.81680
Step [30/31], Loss: 12.47634
Time elasped: 56.55072069168091
Epoch [5/50]
Step [10/31], Loss: 11.06387
Step [20/31], Loss: 10.12393
Step [30/31], Loss: 9.34140
Time elasped: 58.256

In [16]:
hidden = (8, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=8, bias=False)
  (f1_f2): Linear(in_features=400, out_features=8, bias=False)
  (f2_o): Linear(in_features=400, out_features=20, bias=False)
)
Total params: 16800
Total mult-adds (M): 4.2
training shd250_8x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.95721
Step [20/31], Loss: 28.65709
Step [30/31], Loss: 25.58072
Time elasped: 59.0331597328186
Epoch [2/50]
Step [10/31], Loss: 21.27088
Step [20/31], Loss: 17.12039
Step [30/31], Loss: 13.57908
Time elasped: 57.52118468284607
Epoch [3/50]
Step [10/31], Loss: 11.31805
Step [20/31], Loss: 9.27189
Step [30/31], Loss: 8.09292
Time elasped: 55.58344030380249
Epoch [4/50]
Step [10/31], Loss: 7.55698
Step [20/31], Loss

In [17]:
hidden = (8, 2, 'f')
depth, stride = 150, 15
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 15, 30, 45, 60, 75, 90, 105, 120, 135]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=8, bias=False)
  (f1_f2): Linear(in_features=80, out_features=8, bias=False)
  (f2_o): Linear(in_features=80, out_features=20, bias=False)
)
Total params: 7840
Total mult-adds (M): 1.96
training shd250_8x2_d_150_15_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.85952
Step [20/31], Loss: 29.20608
Step [30/31], Loss: 27.78124
Time elasped: 63.596545457839966
Epoch [2/50]
Step [10/31], Loss: 25.65355
Step [20/31], Loss: 22.94158
Step [30/31], Loss: 19.70351
Time elasped: 56.53153204917908
Epoch [3/50]
Step [10/31], Loss: 15.77767
Step [20/31], Loss: 13.80026
Step [30/31], Loss: 12.09970
Time elasped: 56.82212257385254
Epoch [4/50]
Step [10/31], Loss: 10.91036
Step [20/31], Loss: 10.13772
Step [30/31], Loss: 9.41400
Time elasped: 56.07917022705078
Epoch [5/50]
Step [10/31], Loss: 8.60693
Step [20/31], Loss: 8.21693
Step [30/31], Loss: 7.5622

In [18]:
hidden = (8, 2, 'f')
depth, stride = 150, 30
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 30, 60, 90, 120]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=8, bias=False)
  (f1_f2): Linear(in_features=40, out_features=8, bias=False)
  (f2_o): Linear(in_features=40, out_features=20, bias=False)
)
Total params: 6720
Total mult-adds (M): 1.68
training shd250_8x2_d_150_30_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.97371
Step [20/31], Loss: 29.76703
Step [30/31], Loss: 29.14154
Time elasped: 60.071614265441895
Epoch [2/50]
Step [10/31], Loss: 28.23745
Step [20/31], Loss: 27.07428
Step [30/31], Loss: 25.78134
Time elasped: 56.10231828689575
Epoch [3/50]
Step [10/31], Loss: 23.98254
Step [20/31], Loss: 21.99726
Step [30/31], Loss: 19.99112
Time elasped: 54.76580190658569
Epoch [4/50]
Step [10/31], Loss: 18.55698
Step [20/31], Loss: 16.61080
Step [30/31], Loss: 15.49979
Time elasped: 54.99551177024841
Epoch [5/50]
Step [10/31], Loss: 14.33112
Step [20/31], Loss: 13.35293
Step [30/31], Loss: 12.11365
Time elasped: 54

In [19]:
hidden = (4, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=4, bias=False)
  (f1_f2): Linear(in_features=200, out_features=4, bias=False)
  (f2_o): Linear(in_features=200, out_features=20, bias=False)
)
Total params: 7600
Total mult-adds (M): 1.9
training shd250_4x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.84372
Step [20/31], Loss: 29.20317
Step [30/31], Loss: 26.48111
Time elasped: 54.82384634017944
Epoch [2/50]
Step [10/31], Loss: 22.91104
Step [20/31], Loss: 19.76996
Step [30/31], Loss: 17.40175
Time elasped: 57.87751317024231
Epoch [3/50]
Step [10/31], Loss: 14.71744
Step [20/31], Loss: 13.40546
Step [30/31], Loss: 12.70974
Time elasped: 55.821011543273926
Epoch [4/50]
Step [10/31], Loss: 11.48239
Step [20/31], 

In [21]:
hidden = (4, 2, 'f')
depth, stride = 150, 15
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 15, 30, 45, 60, 75, 90, 105, 120, 135]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=4, bias=False)
  (f1_f2): Linear(in_features=40, out_features=4, bias=False)
  (f2_o): Linear(in_features=40, out_features=20, bias=False)
)
Total params: 3760
Total mult-adds (M): 0.94
training shd250_4x2_d_150_15_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.97635
Step [20/31], Loss: 29.68366
Step [30/31], Loss: 29.19556
Time elasped: 55.04518985748291
Epoch [2/50]
Step [10/31], Loss: 28.67076
Step [20/31], Loss: 28.45134
Step [30/31], Loss: 27.72233
Time elasped: 57.677212715148926
Epoch [3/50]
Step [10/31], Loss: 27.00384
Step [20/31], Loss: 25.66718
Step [30/31], Loss: 23.99769
Time elasped: 54.16377544403076
Epoch [4/50]
Step [10/31], Loss: 21.90433
Step [20/31], Loss: 20.02733
Step [30/31], Loss: 18.46313
Time elasped: 55.94419717788696
Epoch [5/50]
Step [10/31], Loss: 16.87556
Step [20/31], Loss: 16.05100
Step [30/31], Loss: 15.

In [22]:
hidden = (4, 2, 'f')
depth, stride = 150, 30
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 30, 60, 90, 120]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=4, bias=False)
  (f1_f2): Linear(in_features=20, out_features=4, bias=False)
  (f2_o): Linear(in_features=20, out_features=20, bias=False)
)
Total params: 3280
Total mult-adds (M): 0.82
training shd250_4x2_d_150_30_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 29.85384
Step [20/31], Loss: 29.66383
Step [30/31], Loss: 29.15747
Time elasped: 53.71144962310791
Epoch [2/50]
Step [10/31], Loss: 28.57595
Step [20/31], Loss: 27.95152
Step [30/31], Loss: 27.21994
Time elasped: 53.33319664001465
Epoch [3/50]
Step [10/31], Loss: 26.59330
Step [20/31], Loss: 25.86795
Step [30/31], Loss: 25.42045
Time elasped: 54.37263321876526
Epoch [4/50]
Step [10/31], Loss: 24.80396
Step [20/31], Loss: 24.12247
Step [30/31], Loss: 23.43109
Time elasped: 54.01409196853638
Epoch [5/50]
Step [10/31], Loss: 22.53256
Step [20/31], Loss: 22.11474
Step [30/31], Loss: 21.23584
Time elasped: 55.